In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from scipy.stats import boxcox, boxcox_normmax
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold, cross_val_score
from sklearn.metrics import mean_absolute_error , r2_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression , Ridge , Lasso, RidgeCV
from sklearn.preprocessing import OneHotEncoder ,RobustScaler


In [ ]:

pd.set_option ('display.max_columns', None)
# pd.set_option ('display.max_rows', None)
#pd.set_option ('max_rows', 400)
pd.set_option ('max_colwidth', 400)

In [ ]:
df_train = pd.read_csv(r"/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv(r"/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
df_train

In [ ]:
df_train.describe()

In [ ]:
df_train.head(5)

# Data Wrangling


In [ ]:
(df_train.isnull().sum()/1460 *100).sort_values().tail(20)


In [ ]:
df_train.drop(["PoolQC","Fence","Alley","MiscFeature"],axis=1,inplace=True)

In [ ]:
df_train["GarageYrBlt"] = df_train["GarageYrBlt"].fillna(0)

In [ ]:
df_train["MasVnrArea"] = df_train["MasVnrArea"].fillna(0)

In [ ]:
df_train["LotFrontage"] = df_train["LotFrontage"].fillna(0)

In [ ]:
df_train["GarageYrBlt"] = df_train["GarageYrBlt"].astype(int)

In [ ]:
df_train["MasVnrArea"] = df_train["MasVnrArea"].astype(int)

In [ ]:
df_train["LotFrontage"] = df_train["LotFrontage"].astype(int)

In [ ]:
df_train["Electrical"].describe(include = 'all')

In [ ]:
df_train["Electrical"].value_counts()

In [ ]:
df_train["Electrical"] = df_train["Electrical"].fillna("SBrkr")

In [ ]:
df_train = df_train.fillna("None")

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8);

In [ ]:
cor = df_train.drop(columns= 'SalePrice').select_dtypes(include='number').copy()

In [ ]:
corr = cor.corr()

fig , ax = plt.subplots(figsize=(25 , 20))


sns.heatmap(corr ,annot= True , ax=ax , cmap= 'Blues');

In [ ]:
df_train.columns

In [ ]:
df_train['GarageArea'].corr(df_train['GarageCars'])

In [ ]:
print(df_train['GarageArea'].corr(df_train['SalePrice']))
print(df_train['GarageCars'].corr(df_train['SalePrice']))

In [ ]:
df_train['GrLivArea'].corr(df_train['2ndFlrSF'])

In [ ]:
print(df_train['GrLivArea'].corr(df_train['SalePrice']))
print(df_train['2ndFlrSF'].corr(df_train['SalePrice']))

In [ ]:
print(df_train['GrLivArea'].corr(df_train['SalePrice']))
print(df_train['TotRmsAbvGrd'].corr(df_train['SalePrice']))

In [ ]:
print(df_train['GrLivArea'].corr(df_train['SalePrice']))
print(df_train['FullBath'].corr(df_train['SalePrice']))

In [ ]:
df_train['BsmtFullBath'].corr(df_train['BsmtFinSF1'])

In [ ]:
print(df_train['BsmtFullBath'].corr(df_train['SalePrice']))
print(df_train['BsmtFinSF1'].corr(df_train['SalePrice']))

In [ ]:
df_train.drop(["GarageArea","BsmtFullBath","FullBath","TotRmsAbvGrd","2ndFlrSF","GarageYrBlt",'1stFlrSF'],axis=1,inplace=True)

In [ ]:
df_train.describe()

In [ ]:
df2 = df_train.drop(columns= 'SalePrice')

In [ ]:
df_numeric = df2.select_dtypes(include='number').copy()
fig, axs = plt.subplots(10, 4,figsize = (35,35))
# Flatten the list of lists of subplots into a single list
axs = axs.ravel()
# Plot the boxplots for each column on the corresponding subplot
for i, column in enumerate(df_numeric):
  sns.boxplot(data=df_numeric[column], ax=axs[i])
  axs[i].set_title(column)
# Adjust the spacing between the subplots


In [ ]:
skew_features = df2[df_numeric.columns].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

In [ ]:
skew_index

In [ ]:
df_train


In [ ]:
for i in skew_index:
    initial_lambda = 0.15
    df_train[i]  = boxcox(df_train[i]+1, lmbda=initial_lambda)

In [ ]:
df_numeric = df2.select_dtypes(include='number').copy()

fig, axs = plt.subplots(10, 4,figsize = (35,35))
# Flatten the list of lists of subplots into a single list
axs = axs.ravel()
# Plot the boxplots for each column on the corresponding subplot
for i, column in enumerate(df_numeric):
  sns.boxplot(data=df_numeric[column], ax=axs[i])
  axs[i].set_title(column)
# Adjust the spacing between the subplots


In [ ]:
df_train.select_dtypes(include='object').columns

In [ ]:
cols = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
df_train[cols] = df_train[cols].apply(LabelEncoder().fit_transform)

In [ ]:
corr = df_train.drop(columns= 'SalePrice').corr()

fig , ax = plt.subplots(figsize=(45 , 40))


sns.heatmap(corr ,annot= True , ax=ax , cmap= 'Blues');

In [ ]:
target = 'SalePrice'
X = df_train.drop(columns=[target])
y = df_train[target]

In [ ]:
X_train , X_test , y_train , y_test = train_test_split(X ,y ,test_size=0.25 , random_state=42 )

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
Lrg = LinearRegression()

Lrg.fit(X_train , y_train);

In [ ]:
mean_absolute_error(y_train , Lrg.predict(X_train))

In [ ]:
reg_y_pred_train = Lrg.predict(X_train)

In [ ]:
reg_y_pred_test = Lrg.predict(X_test)

reg_acc_train = r2_score(y_train , reg_y_pred_train)
reg_acc_test = r2_score(y_test , reg_y_pred_test)

print("Training Accuracy:", round(reg_acc_train, 4))
print("Test Accuracy:", round(reg_acc_test, 4))

In [ ]:
rge = Ridge()
rge.fit(X_train , y_train);

In [ ]:
mean_absolute_error(y_train , rge.predict(X_train))

In [ ]:
rge_y_pred_test = rge.predict(X_test)
rge_y_pred_train = rge.predict(X_train)


rge_acc_train = r2_score(y_train , rge_y_pred_train)
rge_acc_test = r2_score(y_test , rge_y_pred_test)

print("Training Accuracy:", round(rge_acc_train, 4))
print("Test Accuracy:", round(rge_acc_test, 4))

In [ ]:
lso =  make_pipeline(RobustScaler(), Lasso(alpha =0.0005))

lso.fit(X_train , y_train);

In [ ]:
mean_absolute_error(y_train , lso.predict(X_train))

In [ ]:
lso_y_pred_test = lso.predict(X_test)
lso_y_pred_train = lso.predict(X_train)


lso_acc_train = r2_score(y_train , lso_y_pred_train)
lso_acc_test = r2_score(y_test , lso_y_pred_test)

print("Training Accuracy:", round(lso_acc_train, 4))
print("Test Accuracy:", round(lso_acc_test, 4))

In [ ]:
models = pd.DataFrame({

    'Models': ['LinearRegression' , 'Ridge' , 'Lasso'],
    'Scores': [reg_acc_test, rge_acc_test , lso_acc_test]

})

models.sort_values(by='Scores', ascending= False)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
kn = KNeighborsRegressor(n_neighbors=50)
kn.fit(X_train,y_train)


In [ ]:
print(kn.score(X_train,y_train))
print(kn.score(X_test,y_test))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators=1200,
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          random_state=42)

In [ ]:
rf.fit(X_train , y_train);

In [ ]:
print (rf.score(X_train , y_train))
print (rf.score(X_test , y_test))

In [ ]:
mean_absolute_error(y_train , rf.predict(X_train))

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = DecisionTreeRegressor(
                          max_depth=15,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          random_state=42)
dt.fit(X_train , y_train);

In [ ]:
print (dt.score(X_train , y_train))
print (dt.score(X_test , y_test))

In [ ]:
mean_absolute_error(y_train , dt.predict(X_train))

In [ ]:
from xgboost import XGBRegressor

In [ ]:
xgb = XGBRegressor(n_estimators= 1000 , max_depth= 3 , learning_rate = 0.01)
xgb.fit(X_train , y_train);

In [ ]:
print (xgb.score(X_train , y_train))
print (xgb.score(X_test , y_test))

In [ ]:
mean_absolute_error(y_train , xgb.predict(X_train))

In [ ]:
df_test["Electrical"] = df_test["Electrical"].fillna("SBrkr")

In [ ]:
df_test.drop(["GarageArea","BsmtFullBath","FullBath","TotRmsAbvGrd","2ndFlrSF","GarageYrBlt",'1stFlrSF'],axis=1,inplace=True)

In [ ]:
df_test.drop(["PoolQC","Fence","Alley","MiscFeature"],axis=1,inplace=True)

In [ ]:
df_test["MasVnrArea"] = df_test["MasVnrArea"].fillna(0)

In [ ]:
df_test["LotFrontage"] = df_test["LotFrontage"].fillna(0)

In [ ]:
df_test["MasVnrArea"] = df_test["MasVnrArea"].astype(int)

In [ ]:
df_test["LotFrontage"] = df_test["LotFrontage"].astype(int)

In [ ]:
cols = ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'SaleType', 'SaleCondition']
df_test[cols] = df_test[cols].apply(LabelEncoder().fit_transform)


In [ ]:
predections = xgb.predict(df_test)

In [ ]:
predections

In [ ]:
submisson = df_test[["Id"]]

In [ ]:
submisson["SalePrice"] = predections

In [ ]:
submisson

In [ ]:
submisson.to_csv("submisson.csv" , index = False)